<a href="https://colab.research.google.com/github/shanjiang1994/GIS/blob/main/Convert_RGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# !pip install sentinelsat
# !pip install geopandas
# !pip install rasterio

     |████████████████████████████████| 1.0MB 30.3MB/s 
     |████████████████████████████████| 15.4MB 230kB/s 
     |████████████████████████████████| 6.6MB 40.6MB/s 


In [19]:
from sentinelsat import SentinelAPI
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import getpass

import geopandas as gpd
# import folium 
import rasterio as rio

from google.colab import drive
drive.mount('/content/drive')

In [23]:
user = input("Username:")
password = getpass.getpass('Password:')
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')
user,password = '','' # remove variables

Username:sjiang32
Password:··········


In [62]:
nReserve = gpd.read_file('/content/drive/MyDrive/NYCdata/NYC_Shape.shp')

In [148]:
# nyc = folium.Map(location=[40.8075,-73.9857],zoom_start=13)
# folium.GeoJson(nReserve).add_to(nyc)
# nyc

In [149]:
# from shapely.geometry import MultiPolygon, Polygon
# ft = None
# for i in nReserve['geometry']:
#   ft = i
# ft

In [107]:
footprint = geojson_to_wkt(read_geojson('/content/drive/MyDrive/NYCdata/nyc_shape.json'))

In [108]:
footprint

'POLYGON((-73.9103 40.9343,-73.9190 40.9249,-73.9281 40.8920,-73.9500 40.8541,-73.9668 40.8202,-73.9936 40.7864,-74.0111 40.7629,-74.0192 40.7351,-74.0304 40.6935,-74.0548 40.6539,-74.0746 40.6521,-74.0850 40.6500,-74.0939 40.6477,-74.1035 40.6477,-74.1144 40.6461,-74.1229 40.6427,-74.1309 40.6432,-74.1473 40.6423,-74.1624 40.6465,-74.1763 40.6477,-74.1848 40.6458,-74.1957 40.6368,-74.2017 40.6312,-74.2025 40.6223,-74.2035 40.6148,-74.2032 40.6052,-74.1994 40.6005,-74.2002 40.5970,-74.2049 40.5907,-74.2073 40.5834,-74.2094 40.5741,-74.2150 40.5607,-74.2013 40.5604,-74.1276 40.5598,-74.0614 40.5593,-73.9699 40.5586,-73.9033 40.5577,-73.8212 40.5569,-73.8162 40.5701,-73.8096 40.5894,-73.8062 40.5978,-73.8004 40.6140,-73.7940 40.6316,-73.7873 40.6508,-73.7797 40.6717,-73.7724 40.6913,-73.7663 40.7087,-73.7589 40.7298,-73.7501 40.7539,-73.7340 40.7984,-73.7212 40.8333,-73.7066 40.8738,-73.7071 40.8740,-73.9103 40.9343))'

In [133]:
# querying the data
products = api.query(footprint,
                     date = ('20191101', '20201101'),
                     platformname = 'Sentinel-2',
                    #  processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,0.5)
                    )

In [134]:
products_df = api.to_dataframe(products)
products_df.head

,title,link,link_alternative,link_icon,summary,ondemand,datatakesensingstart,beginposition,endposition,ingestiondate,orbitnumber,relativeorbitnumber,cloudcoverpercentage,sensoroperationalmode,gmlfootprint,footprint,level1cpdiidentifier,tileid,hv_order_tileid,format,processingbaseline,platformname,filename,instrumentname,instrumentshortname,size,s2datatakeid,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,identifier,uuid,granuleidentifier,datastripidentifier,vegetationpercentage,notvegetatedpercentage,waterpercentage,unclassifiedpercentage,mediumprobacloudspercentage,highprobacloudspercentage,snowicepercentage
5f3bffd3-461d-4729-8683-3b98a48bfa36,S2A_MSIL1C_20201027T155341_N0209_R054_T18TXK_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-10-27T15:53:41.024Z, Instrument: MS...",false,2020-10-27 15:53:41.024,2020-10-27 15:53:41.024,2020-10-27 15:53:41.024,2020-10-27 19:32:35.182,27936,54,0.000000,INS-NOBS,"<gml:Polygon srsName=""http://www.opengis.net/g...",MULTIPOLYGON (((-73.81869312611339 40.56393318...,S2A_OPER_MSI_L1C_TL_VGS2_20201027T180124_A0279...,18TXK,TK18X,SAFE,02.09,Sentinel-2,S2A_MSIL1C_20201027T155341_N0209_R054_T18TXK_2...,Multi-Spectral Instrument,MSI,20.73 MB,GS2A_20201027T155341_027936_N02.09,S2MSI1C,2015-028A,DESCENDING,Sentinel-2A,Level-1C,S2A_MSIL1C_20201027T155341_N0209_R054_T18TXK_2...,5f3bffd3-461d-4729-8683-3b98a48bfa36,S2A_OPER_MSI_L1C_TL_VGS2_20201027T180124_A0279...,S2A_OPER_MSI_L1C_DS_VGS2_20201027T180124_S2020...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270459d9-252d-4a57-ab86-f42f82843972,S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-10-17T15:52:51.024Z, Instrument: MS...",false,NaT,2020-10-17 15:52:51.024,2020-10-17 15:52:51.024,2020-10-18 00:51:24.071,27793,54,0.421761,NaN,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((-73.8201 40.55612818801274, -7...",S2A_OPER_MSI_L1C_TL_VGS2_20201017T193914_A0277...,NaN,NaN,SAFE,02.14,Sentinel-2,S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_2...,Multi-Spectral Instrument,MSI,1.11 GB,GS2A_20201017T155251_027793_N02.14,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_2...,270459d9-252d-4a57-ab86-f42f82843972,NaN,NaN,79.765254,8.271509,6.210702,1.176771,0.070524,0.309463,0.001543
a0bed006-c610-4353-be31-f6db0165d159,S2A_MSIL2A_20201017T155251_N0214_R054_T18TXL_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-10-17T15:52:51.024Z, Instrument: MS...",false,NaT,2020-10-17 15:52:51.024,2020-10-17 15:52:51.024,2020-10-18 00:50:49.700,27793,54,0.313522,NaN,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((-73.81879 40.55974178921592, -...",S2A_OPER_MSI_L1C_TL_VGS2_20201017T193914_A0277...,NaN,NaN,SAFE,02.14,Sentinel-2,S2A_MSIL2A_20201017T155251_N0214_R054_T18TXL_2...,Multi-Spectral Instrument,MSI,180.84 MB,GS2A_20201017T155251_027793_N02.14,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_MSIL2A_20201017T155251_N0214_R054_T18TXL_2...,a0bed006-c610-4353-be31-f6db0165d159,NaN,NaN,80.326635,4.992813,10.565376,1.141547,0.044723,0.095257,0.002495
01486e68-a0eb-4216-9289-2550510c7bad,S2A_MSIL1C_20201017T155251_N0209_R054_T18TWL_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-10-17T15:52:51.024Z, Instrument: MS...",false,2020-10-17 15:52:51.024,2020-10-17 15:52:51.024,2020-10-17 15:52:51.024,2020-10-17 21:11:53.822,27793,54,0.000000,INS-NOBS,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((-73.8201 40.55612818801274, -7...",S2A_

In [140]:
# Select one online sentinle image as test image.
for i in range(len(products_df.index)):
  product_id = products_df.index[i]
  product_info = api.get_product_odata(product_id)
  if product_info['Online'] == True:
   print(f"Detected {product_id} is online  -> Dataframe index {i}")
   break

270459d9-252d-4a57-ab86-f42f82843972is online -> True, index 1


In [142]:
api.download(products_df.index[1])

{'Creation Date': datetime.datetime(2020, 10, 18, 0, 52, 16, 994000),
 'Ingestion Date': datetime.datetime(2020, 10, 18, 0, 51, 24, 71000),
 'Online': True,
 'date': datetime.datetime(2020, 10, 17, 15, 52, 51, 24000),
 'downloaded_bytes': 1188979744,
 'footprint': 'POLYGON((-73.696594 40.89740435913649,-73.731415 40.80118818167337,-73.78427 40.65507827546429,-73.8201 40.55612818801274,-75.000244 40.56274991351036,-75.000244 41.551844670066835,-73.68381 41.54431153543986,-73.696594 40.89740435913649))',
 'id': '270459d9-252d-4a57-ab86-f42f82843972',
 'md5': '298d90fd945cce7b3eb990fdc8362e87',
 'path': 'S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106.zip',
 'quicklook_url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('270459d9-252d-4a57-ab86-f42f82843972')/Products('Quicklook')/$value",
 'size': 1188979744,
 'title': 'S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('270459d9-252d-4a57-ab86-f

In [147]:
# unzip the file
%%bash
mv /content/S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106.zip /content/drive/MyDrive/S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106.zip
cd /content/drive/MyDrive
unzip -d /content/drive/MyDrive/S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106 S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106.zip

Archive:  S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106.zip
   creating: /content/drive/MyDrive/S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106/S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106.SAFE/
 extracting: /content/drive/MyDrive/S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106/S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106.SAFE/INSPIRE.xml  
   creating: /content/drive/MyDrive/S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106/S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106.SAFE/rep_info/
 extracting: /content/drive/MyDrive/S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106/S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106.SAFE/rep_info/S2_PDI_Level-2A_Tile_Metadata.xsd  
 extracting: /content/drive/MyDrive/S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106/S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106.SAFE/rep_info/S2_PDI_Level-2A_Datastrip_Metad

In [158]:
# open different bands files and write to RGB.tif file

R10 = '/content/drive/MyDrive/S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106/S2A_MSIL2A_20201017T155251_N0214_R054_T18TWL_20201017T201106.SAFE/GRANULE/L2A_T18TWL_A027793_20201017T155552/IMG_DATA/R10m'
b4 = rio.open(R10+'/T18TWL_20201017T155251_B04_10m.jp2')
b3 = rio.open(R10+'/T18TWL_20201017T155251_B03_10m.jp2')
b2 = rio.open(R10+'/T18TWL_20201017T155251_B02_10m.jp2')

file_name = '10m_RGB'
with rio.open(os.path.join(R10,f'{file_name}.tif'),'w',driver='Gtiff', width=b4.width, height=b4.height, 
              count=3,crs=b4.crs,transform=b4.transform, dtype=b4.dtypes[0]) as rgb:
    rgb.write(b2.read(1),1) 
    rgb.write(b3.read(1),2) 
    rgb.write(b4.read(1),3) 
    rgb.close()